## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation
st = study('Olfacto')
conds, subjects = ['low','high'],['FERJ','PIRJ']#,'CHAF','VACJ','SEMC','LEFC']
freqs = ['0_theta','1_alpha','2_beta','3_gamma']
path_pow = path.join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_cond/TPS_R_p_by_cond/')
save_path = path.join(st.path, 'classified/Classif_TPSim_E_R_all_BBG_within/')
nperm = 1000

for su in subjects:
    for freq in freqs:
        pow_list = []
        #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
        mat0 = np.load(path.join(path_pow, 'TPS_spear_'+su+'_'+conds[0]+'_'+freq+'.npz'),
                       allow_pickle=True)
        names = mat0['label']
        pow_list.append(mat0['tps'])
        nelecs = mat0['tps'].shape[0]
        mat1 = np.load(path.join(path_pow, 'TPS_spear_'+su+'_'+conds[1]+'_'+freq+'.npz'),
                       allow_pickle=True)
        pow_list.append(mat1['tps'])
        print (su, 'TPS shape: ', [pow.shape for pow in pow_list])
        # =========================== Select Power for 1 elec 1 freq =================================                 
        iterator = range(nelecs)
        for elec_num in iterator:#iterator
            elec_label = names[elec_num]
            print ('elec ', elec_num, 'elec_label ', elec_label)
            #Filenames to save
            name_auc = (save_path+freq+'/'+su +'_auc_'+conds[0]+'_'+conds[1]+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
            name_perm = (save_path+freq+'/'+su +'_perm_'+str(elec_label)+'_('+str(elec_num)+').npy')
                        
            if path.exists(name_auc):
                print(su,phase,elec_num,freq,'already computed')
            else:
                print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                pow_data_elec = []
                for i,power in enumerate(pow_list):
                    pow_data_elec.append(power[elec_num][:,np.newaxis])
        # =============================  Classification Computation ============================================================           
                # create a data matrix, concatenate along the trial dimension
                x = np.concatenate(pow_data_elec, axis=0)
                print ('Size of the concatenated data: ', x.shape, 'Number time windows : ', x.shape[1])
                y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                print ('Size of label for classif: ', len(y))

                auc = np.array([])
                for t in range(x.shape[1]):
                    X = x[:,t]
                    X = X.reshape(-1, 1)
                    score_rep = []
                    for i in range(10):
                        k = 5
                        skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                        skf.get_n_splits(X, y)
                        score_cv = []
                        for train_index, test_index in skf.split(X, y):
                            clf = LDA()
                            X_train, X_test = X[train_index], X[test_index]
                            y_train, y_test = y[train_index], y[test_index]
                            clf.fit(X=X_train, y=y_train)
                            y_pred = clf.predict(X_test)
                            score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                        score_rep.append(np.mean(score_cv))
                    score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                    auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                auc = np.swapaxes(auc,0,1)

                perm_scores = np.array([])
                for t in range(x.shape[1]):
                    X = x[:,t]
                    X = X.reshape(-1, 1)
                    perm_rep = []
                    for perm in range(nperm):
                        y_perm = y[permutation(len(y))]
                        score_cv = []
                        for train_index, test_index in skf.split(X, y_perm):
                            clf = LDA()
                            X_train, X_test = X[train_index], X[test_index]
                            y_train, y_test = y_perm[train_index], y_perm[test_index]
                            clf.fit(X=X_train, y=y_train)
                            y_pred = clf.predict(X_test)
                            score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                        perm_rep.append(np.mean(score_cv))
                    perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                    perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                perm_scores = np.swapaxes(perm_scores,0,1)           
                th_0_05_perm = perm_pvalue2level(perm_scores, p=0.005, maxst=True)
                th_0_01_perm = perm_pvalue2level(perm_scores, p=0.001, maxst=True)
                print('th_perm 005: ', th_0_05_perm[0], '001',th_0_01_perm[0], 'auc_max', np.max(auc))

                #Save plots
                np.save(name_auc, auc)
                np.save(name_perm, perm_scores)
                del X, auc, pow_data_elec
    del pow_list

-> Olfacto loaded
FERJ TPS shape:  [(45, 204), (45, 168)]
elec  0 elec_label  Amg
--» processing FERJ elec 0 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.548533976783618 001 0.5590902569453503 auc_max 0.5071540367810095
elec  1 elec_label  Amg
--» processing FERJ elec 1 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5588571149080475 001 0.5913212469023087 auc_max 0.6332369244815442
elec  2 elec_label  HC
--» processing FERJ elec 2 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5683003999826094 001 0.595920394765445 auc_max 0.6815040650406504
elec  3 elec_label  HC
--» processing FERJ elec 3 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5614442850310

th_perm 005:  0.5618722012086431 001 0.5737246206686666 auc_max 0.49278922655536717
elec  34 elec_label  OFC
--» processing FERJ elec 34 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5635522368592671 001 0.5688822225120647 auc_max 0.5146662101647754
elec  35 elec_label  OFC
--» processing FERJ elec 35 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5600291291682972 001 0.5801174948915264 auc_max 0.4998491369940437
elec  36 elec_label  OFC
--» processing FERJ elec 36 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5566268857875745 001 0.5756714925438023 auc_max 0.503030303030303
elec  37 elec_label  OFC
--» processing FERJ elec 37 / 45 freq 0_theta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classi

th_perm 005:  0.5657589887396199 001 0.5939471109951742 auc_max 0.5516962305986697
elec  22 elec_label  Ins
--» processing FERJ elec 22 / 45 freq 1_alpha
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5641178644406764 001 0.5792075344550237 auc_max 0.5
elec  23 elec_label  Ins
--» processing FERJ elec 23 / 45 freq 1_alpha
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5539120038259207 001 0.565885396287118 auc_max 0.6142440546063215
elec  24 elec_label  pPirT
--» processing FERJ elec 24 / 45 freq 1_alpha
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5564401547758793 001 0.5642199252206426 auc_max 0.5920794748054432
elec  25 elec_label  pPirT
--» processing FERJ elec 25 / 45 freq 1_alpha
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_p

th_perm 005:  0.5678180296508848 001 0.5810911482109474 auc_max 0.5035324551106474
elec  10 elec_label  HC
--» processing FERJ elec 10 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5668614408069214 001 0.5828696578409633 auc_max 0.5945253467240555
elec  11 elec_label  HC
--» processing FERJ elec 11 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5598553323768531 001 0.5864666753619409 auc_max 0.6182460545193688
elec  12 elec_label  PHG
--» processing FERJ elec 12 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5555296508847442 001 0.5680854093300292 auc_max 0.5072612060345203
elec  13 elec_label  PHG
--» processing FERJ elec 13 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  37

th_perm 005:  0.5585966914481979 001 0.5641636233207252 auc_max 0.503780270423025
elec  43 elec_label  Ins
--» processing FERJ elec 43 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.552619886091909 001 0.5719652406417112 auc_max 0.5
elec  44 elec_label  Ins
--» processing FERJ elec 44 / 45 freq 2_beta
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5655504108517021 001 0.5868995913221164 auc_max 0.6329563932002957
FERJ TPS shape:  [(45, 204), (45, 168)]
elec  0 elec_label  Amg
--» processing FERJ elec 0 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.556940133037694 001 0.587573366375375 auc_max 0.5178184644145907
elec  1 elec_label  Amg
--» processing FERJ elec 1 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of l

th_perm 005:  0.5667527498804399 001 0.5780854093300292 auc_max 0.5
elec  31 elec_label  PHG
--» processing FERJ elec 31 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5609374592409025 001 0.5746026259727838 auc_max 0.5683870266510151
elec  32 elec_label  PHG
--» processing FERJ elec 32 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5628985696274075 001 0.5789963479848702 auc_max 0.5613350506499718
elec  33 elec_label  IFG
--» processing FERJ elec 33 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.5509991956871441 001 0.5713311377766184 auc_max 0.5
elec  34 elec_label  OFC
--» processing FERJ elec 34 / 45 freq 3_gamma
Size of the concatenated data:  (372, 1) Number time windows :  1
Size of label for classif:  372
th_perm 005:  0.557348

th_perm 005:  0.5754095577624989 001 0.5875999490705374 auc_max 0.5483248450895509
elec  19 elec_label  IFG
--» processing PIRJ elec 19 / 30 freq 0_theta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5685862829980477 001 0.5974577709871827 auc_max 0.536840675664205
elec  20 elec_label  IFG
--» processing PIRJ elec 20 / 30 freq 0_theta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5770626432391138 001 0.5918818436465495 auc_max 0.4987369493251846
elec  21 elec_label  OFC
--» processing PIRJ elec 21 / 30 freq 0_theta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5751663695781344 001 0.5804638825227059 auc_max 0.5058747135217724
elec  22 elec_label  OFC
--» processing PIRJ elec 22 / 30 freq 0_theta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif

th_perm 005:  0.5765580171462525 001 0.5890896358543418 auc_max 0.5669930396400985
elec  22 elec_label  OFC
--» processing PIRJ elec 22 / 30 freq 1_alpha
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5771513453866396 001 0.5899902385196503 auc_max 0.49828749681690854
elec  23 elec_label  OFC
--» processing PIRJ elec 23 / 30 freq 1_alpha
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5715469824293353 001 0.5839635854341736 auc_max 0.4950148544266191
elec  24 elec_label  OFC
--» processing PIRJ elec 24 / 30 freq 1_alpha
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5734776334776335 001 0.5883307868601986 auc_max 0.48958747135217723
elec  25 elec_label  OFC
--» processing PIRJ elec 25 / 30 freq 1_alpha
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for clas

th_perm 005:  0.5801922587216705 001 0.5859952465834819 auc_max 0.6594011544011543
elec  25 elec_label  OFC
--» processing PIRJ elec 25 / 30 freq 2_beta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5748395721925134 001 0.6005576776165011 auc_max 0.47092691622103383
elec  26 elec_label  OFC
--» processing PIRJ elec 26 / 30 freq 2_beta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5770910788557846 001 0.5872277395806806 auc_max 0.6008148714031067
elec  27 elec_label  Ins
--» processing PIRJ elec 27 / 30 freq 2_beta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5737110601816484 001 0.6021326712503183 auc_max 0.5786066547831255
elec  28 elec_label  Ins
--» processing PIRJ elec 28 / 30 freq 2_beta
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif: 

th_perm 005:  0.5775273745861981 001 0.5892093200916729 auc_max 0.5128927934810288
elec  28 elec_label  Ins
--» processing PIRJ elec 28 / 30 freq 3_gamma
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5681792717086835 001 0.5830999066293184 auc_max 0.5268992445463033
elec  29 elec_label  Ins
--» processing PIRJ elec 29 / 30 freq 3_gamma
Size of the concatenated data:  (344, 1) Number time windows :  1
Size of label for classif:  344
th_perm 005:  0.5701061030472795 001 0.5797907647907647 auc_max 0.5686894151600034
